# Amazon Bestselling Snack Products - Data Processing

In [4]:
import csv
import re
from collections import Counter

#### Get raw data

In [5]:
with open('amazon_snacks_scraped_data.csv', newline='', encoding='utf-8') as datafile:
    reader = csv.DictReader(datafile)

    raw_data = []
    for row in reader:
        raw_data.append(row)

#### Clean data

In [6]:
class data_cleaner:
    def __init__(self, raw_row):
        self.raw_row = raw_row
        self.clean_row = {}

    def add_full_title(self):
        self.clean_row.update({'title': self.raw_row['title']})

    def add_simplified_title(self):
        raw_title = self.raw_row['title']
        
        if raw_title.find('|') > -1:
            simplified_title = raw_title.split('|')[0]
        elif raw_title.find(' - ') > -1:
            simplified_title = raw_title.split(' - ')[0]
        elif raw_title.find(',') > -1:
            simplified_title = raw_title.split(',')[0]
        elif raw_title.find(';') > -1:
            simplified_title = raw_title.split(';')[0]
        else:
            simplified_title = raw_title
        self.clean_row.update({'simplified_title': simplified_title})

    def add_list_price(self):
        self.clean_row.update({'list_price': self.raw_row['list_price']})

    def add_sale_price(self):
        self.clean_row.update({'sale_price': self.raw_row['sale_price']})

    def add_has_deal(self):
        self.clean_row.update({'has_deal': self.raw_row['has_deal']})

    def add_deal_discount(self):
        if self.clean_row['has_deal'] == 'TRUE':
            deal_discount = round((1 - float(self.clean_row['sale_price']) / float(self.clean_row['list_price'])) * 100)
            self.clean_row.update({'deal_discount': deal_discount})
        else:
            self.clean_row.update({'deal_discount': ''})

    def add_has_coupon(self):
        self.clean_row.update({'has_coupon': self.raw_row['has_coupon']})
        
    def add_rating(self):
        self.clean_row.update({'rating': self.raw_row['rating']})

    def add_reviews_count(self):
        self.clean_row.update({'reviews_count': self.raw_row['reviews_count']})

    def add_bsr(self):
        self.clean_row.update({'bsr': self.raw_row['bsr']})

    def add_category(self):
        category = self.raw_row['categories'].splitlines()[-2]
        self.clean_row.update({'category': category})

    def add_subcategory(self):
        subcategory = self.raw_row['categories'].splitlines()[-1]
        self.clean_row.update({'subcategory': subcategory})

In [7]:
clean_data = []

for raw_row in raw_data:
    cleaner = data_cleaner(raw_row)

    cleaner.add_full_title()
    cleaner.add_simplified_title()
    cleaner.add_list_price()
    cleaner.add_sale_price()
    cleaner.add_has_deal()
    cleaner.add_has_coupon()
    cleaner.add_deal_discount()
    cleaner.add_rating()
    cleaner.add_reviews_count()
    cleaner.add_bsr()
    cleaner.add_category()
    cleaner.add_subcategory()
    
    clean_data.append(cleaner.clean_row)

#### Write cleaned data

In [15]:
with open('clean_data\cleaned_amazon_snacks_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fields = [
        'bsr',
        'title',
        'simplified_title',
        'list_price',
        'sale_price',
        'has_coupon',
        'has_deal',
        'deal_discount',
        'rating',
        'reviews_count',
        'category',
        'subcategory']
    writer = csv.DictWriter(csvfile, fields)

    writer.writeheader()
    for row in clean_data:
        writer.writerow(row)

#### Analyze title word counts

In [9]:
def get_words(title):
    useless_words = {'the', 'an', 'a', 'and', 'or', 'of', 'for', 'on', 'to', 'per', 'with', 'by', 'lb',
                     'oz', 'ounce', 'ounces', 'count', 'ct', 'snack', 'snacks', 'may', 'vary', '&', '-', '|'}
    title_words = set([word.strip('.,()/*') for word in title.lower().split()])
    filtered_title_words = [word for word in title_words if word not in useless_words]
    return filtered_title_words

In [22]:
word_counts = {}
for row in clean_data:
    category = row['category']
    
    if not word_counts.get(category):
        word_counts.update({category: Counter()})

    title_words = get_words(row['title'])
    word_counts[category].update(title_words)

#### Write title word counts

In [23]:
with open('clean_data\\title_word_counts.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fields = ['category', 'word', 'count']
    writer = csv.DictWriter(csvfile, fields, restval=0)

    for category, word_count in word_counts.items():
        for word, ct in word_count.items():
            row = {'category': category, 'word': word, 'count': ct}
            writer.writerow(row)